# Iris Dataset (builtin dataset)
•The dataset contains a set of 150 records under five attributes sepal
length, sepal width, petal length, petal width and species.

•This data sets consists of 3 different types of irises’ ( Setosa , Versicolour , and Virginica) petal and sepal length, stored in a 150x4 numpy.ndarray.

•The rows being the samples and the columns being: Sepal Length, Sepal Width,
Petal Length and Petal Width.

In [1]:
import sklearn as sk

In [3]:
from sklearn.datasets  import load_iris
iris = load_iris()